<a href="https://colab.research.google.com/github/yoshihiroiida/yoshihiro/blob/main/Bilingual_Kokkai_Explorer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install deep_translator

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 kB 3.1 MB/s eta 0:00:00


In [3]:
"""
【日本語】
このプログラムは以下の手順を行います。
1. ユーザーが英語キーワードを入力すると、それを日本語に翻訳します。
2. 翻訳した日本語キーワードを用いて、国会会議録検索API(Kokkai API)からデータを取得します。
   - 検索期間を（年単位）で指定可能
   - 抽出数（30・50・100）を指定可能
3. 取得した日本語テキストを、再度英語に翻訳して出力します。
4. 検索に使用した日本語キーワードを、取得テキストの中で赤文字にハイライトします。
5. 検索結果をCSVファイルにも保存します。

【English】
This program follows the steps below:
1. The user inputs a keyword in English, which will be translated into Japanese.
2. Using the translated Japanese keyword, data is fetched from the Kokkai API (Japanese Diet minutes).
   - You can specify the search period by year
   - You can choose the number of records to retrieve (30, 50, or 100)
3. The fetched Japanese text is then translated back into English and displayed.
4. The Japanese keyword used for searching is highlighted in red within the retrieved text.
5. The search results are saved to a CSV file.
"""

import requests
import pandas as pd
from deep_translator import GoogleTranslator
from time import sleep
import re

def chunk_text(text, max_length=4500):
    """
    【日本語】
    翻訳APIの制限に対処するため、テキストが長い場合は最大長(max_length)ごとに分割する関数。

    【English】
    Splits the text into chunks that can be processed by the translation API.
    If the text exceeds 'max_length', it's split into multiple parts.

    Args:
        text (str): テキスト (the text to be chunked)
        max_length (int): 各チャンクの最大文字数 (maximum length of each chunk)

    Returns:
        list: 分割されたテキストのリスト (list of text chunks)
    """
    if len(text) <= max_length:
        return [text]

    chunks = []
    current_chunk = ""

    for sentence in text.split("。"):
        if len(current_chunk) + len(sentence) + 1 <= max_length:
            current_chunk += sentence + "。"
        else:
            if current_chunk:
                chunks.append(current_chunk)
            current_chunk = sentence + "。"

    if current_chunk:
        chunks.append(current_chunk)

    return chunks

def translate_text(text, translator):
    """
    【日本語】
    テキストを翻訳する関数。長文の場合は `chunk_text` で分割して、1チャンクごとに翻訳を行う。

    【English】
    Translates the given text using the provided translator object.
    If the text is long, it is split into multiple chunks and translated sequentially.

    Args:
        text (str): 翻訳対象のテキスト (the text to be translated)
        translator (GoogleTranslator): 翻訳に使用するトランスレーターオブジェクト (translator instance)

    Returns:
        str: 翻訳結果 (the translated text)
    """
    chunks = chunk_text(text)
    translated_chunks = []

    for chunk in chunks:
        try:
            translated = translator.translate(chunk)
            translated_chunks.append(translated)
            sleep(1)  # 翻訳APIの制限対策 (delay to avoid rate-limits)
        except Exception as e:
            print(f"Translation error: {e}")
            translated_chunks.append("[Translation Error]")

    return " ".join(translated_chunks)

def highlight_keyword(text, keyword):
    """
    【日本語】
    テキスト中のキーワードを赤字でハイライト表示する関数。
    キーワードが複数回登場する場合も全て置換する。

    【English】
    Highlights the specified keyword in red within the text.
    If the keyword appears multiple times, all occurrences are replaced.

    Args:
        text (str): 検索結果のテキスト (the text to be highlighted)
        keyword (str): ハイライトするキーワード (the keyword to be highlighted)

    Returns:
        str: キーワードが赤字でハイライトされたテキスト (text with the keyword in red)
    """
    pattern = re.compile(re.escape(keyword))
    highlighted_text = pattern.sub(f"\033[31m{keyword}\033[0m", text)
    return highlighted_text

def main():
    """
    【日本語】
    メイン関数:
    1. ユーザーから英語のキーワードを入力
    2. 日本語に翻訳
    3. 年を指定（1年単位）
    4. 抽出数を指定（30・50・100）
    5. 日本語キーワードでAPIを検索
    6. 検索結果を英語に翻訳、キーワードを赤字でハイライトして表示
    7. CSVに保存

    【English】
    Main function:
    1. Prompt the user for a keyword in English
    2. Translate it into Japanese
    3. Specify the year range (in 1-year increments)
    4. Specify the number of records to retrieve (30, 50, or 100)
    5. Search the Kokkai API with the Japanese keyword
    6. Translate the results back to English, highlight the keyword in red, and display
    7. Save the results to a CSV file
    """
    # 1) 英語キーワードの入力 (Input English keyword)
    keyword_en = input("Enter search keyword in English: ")

    # 2) 英語→日本語に翻訳 (Translate English to Japanese)
    translator_en_to_ja = GoogleTranslator(source='en', target='ja')
    keyword_ja = translate_text(keyword_en, translator_en_to_ja)
    print(f"[INFO] Translated keyword (Japanese): {keyword_ja}")

    # 3) 検索期間(年)を選択 (Select the year for the search period)
    year = input("Enter the year (e.g. 2023): ")
    # 年が入力されない、あるいは数字でない場合を簡易的にチェック
    if not year.isdigit():
        print("[WARN] Invalid year. Using default (2023).")
        year = "2023"
    from_date = f"{year}-01-01"
    until_date = f"{year}-12-31"

    # 4) 抽出数を選択 (Choose the number of records to retrieve: 30, 50 or 100)
    record_count = input("Enter the number of records to retrieve (30, 50, 100): ")
    if record_count not in ["30", "50", "100"]:
        print("[WARN] Invalid choice. Using default (100).")
        record_count = "100"

    # 5) Kokkai API を用いた検索 (Search using Kokkai API with the Japanese keyword)
    url = "https://kokkai.ndl.go.jp/api/speech"
    params = {
        "any": keyword_ja,
        "recordPacking": "json",
        "maximumRecords": record_count,
        "from": from_date,
        "until": until_date
    }

    try:
        response = requests.get(url, params=params)
        response.raise_for_status()
        data = response.json()

        if 'speechRecord' not in data or not data['speechRecord']:
            print("[INFO] No data found.")
            return

        # 6) 日本語→英語への翻訳、および赤字ハイライト (Translate results to English & highlight keyword)
        translator_ja_to_en = GoogleTranslator(source='ja', target='en')
        records = []

        for record in data["speechRecord"]:
            if "speech" not in record:
                continue

            original_text = record["speech"]
            # ★ キーワードを赤字ハイライト ★
            highlighted_text = highlight_keyword(original_text, keyword_ja)
            translated_text = translate_text(original_text, translator_ja_to_en)

            # 結果を格納 (Store results)
            records.append({
                'Original (Japanese)': original_text,
                'Highlighted (Japanese)': highlighted_text,
                'Translated (English)': translated_text
            })

            # 結果表示 (Display results)
            print("--------------------------------------------------")
            print("Original (Japanese, highlighted):")
            # 長い場合は200文字まで (Truncate if very long)
            preview_jp = (highlighted_text[:200] + "...") if len(highlighted_text) > 200 else highlighted_text
            print(preview_jp)

            print("Translated (English, preview):")
            preview_en = (translated_text[:200] + "...") if len(translated_text) > 200 else translated_text
            print(preview_en)

        # 7) CSV ファイルに保存 (Save to CSV file)
        if records:
            df = pd.DataFrame(records)
            df.to_csv('translated_sentences.csv', index=False, encoding='utf-8-sig')
            print("\n[INFO] Data saved to 'translated_sentences.csv'.")
        else:
            print("[INFO] No data to save.")

    except requests.exceptions.RequestException as e:
        print(f"[ERROR] API request error: {e}")
    except Exception as e:
        print(f"[ERROR] Unexpected error: {e}")

if __name__ == "__main__":
    main()


Enter search keyword in English: israel
[INFO] Translated keyword (Japanese): イスラエル
Enter the year (e.g. 2023): 2024
Enter the number of records to retrieve (30, 50, 100): 30
Translation error: Request exception can happen due to an api connection error. Please check your connection and try again
--------------------------------------------------
Original (Japanese, highlighted):
○岩屋国務大臣　外務大臣の岩屋毅です。
　堀内委員長を始め、理事、委員各位に外務大臣としての所信を申し述べます。
　ウクライナ侵略が国際秩序を揺るがし、安全保障環境も厳しさを増す中、日米同盟の強化、自由で開かれたインド太平洋の実現に向けた同盟国、同志国との連携、グローバルサウスとの連携の三点を重視し、我が国の平和、地域の安定を実現し、国際社会を分断から協調に導いてまいります。
　私は先月、Ａ...
Translated (English, preview):
[Translation Error]
Translation error: Request exception can happen due to an api connection error. Please check your connection and try again
Translation error: Request exception can happen due to an api connection error. Please check your connection and try again
--------------------------------------------------
Original (Japanese, highlighted):
○野田佳彦君　立憲民主党の野田佳彦です。
　会派を代表して、石破内閣